### Hiệu suất truy vấn không phân cấp

- **Truy vấn 1: Truy vấn những khoá học do website Coursera cung cấp.**
  -   Neo4j:<br>
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)-[:Posted_on]->(w:Website) where w.name = “Coursera” return c
  -   mssql:<br>
wait


- **Truy vấn 2: Thống kê các bài đăng tuyển dụng thuộc về nghiệp Data engineer**
  -   Neo4j:<br>
match(f:FactJobPosting)-[:Belong_to_career]->(c:Career) where c.name = "Data Engineer" return sum(f.totalJobPost) as JobCount
  -   mssql:<br>
wait

### Hiệu suất truy vấn khi phân cấp
- **Truy vấn 3: Truy vấn những khoá học có dạy những kỹ năng chỉ định**
  -   Neo4j:<br>
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]
optional match(fw:Framework)<-[:Have_framework]-(p) where fw.framework in ["SPRING", "SPARK"]
optional match(tl:Tool)<-[:Use_tool]-(p) where p.tool = "GIT"
optional match(kl:Knowledge)<-[:Relate_to_knowledge]-(p) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(pf:Platform)<-[:Deploy_to_platform]-(fw)  where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return count(c)

  -   mssql:<br>
wait

- **Truy vấn 4: Truy vấn những nghề nghiệp có những kỹ năng chỉ định**
  -   Neo4j:<br>
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool)<-[:Use_tool]-(pl) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework)<-[:Have_framework]-(pl) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform)-[Deploy_on_framework]->(fw) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name as CareerName,
    pl.programmingLanguage as ProgrammingLanguage,
    collect(distinct kl.knowledge) as Knowledge,  fw.framework as Framework, collect(distinct tl.tool) as Tool, 
    collect(distinct pf.platform) as Platform

  -   mssql:<br>
wait

### Hiệu suất truy vấn khi phân cấp và tổng hợp
**Truy vấn 5: Tính số lượng các khoá học có dạy những kỹ năng chỉ định và sắp xếp theo thứ tự giảm dần**
  -   Neo4j:<br>
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]
optional match(fw:Framework)<-[:Have_framework]-(p) where fw.framework in ["SPRING", "SPARK"]
optional match(tl:Tool)<-[:Use_tool]-(p) where p.tool = "GIT"
optional match(kl:Knowledge)<-[:Related_to_knowledge]-(p) where p.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(pf:Platform)<-[:Deploy_to_platform]-(fw)  where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return p, tl, kl, fw, pf, count(f) as NumberOfCourse order by NumberOfCourse desc 
  -   mssql:<br>
wait

**Truy vấn 6: Tính số lượng các bài đăng tuyển dụng của các nghề nghiệp có yêu cầu những kỹ năng chỉ định và sắp xếp theo thứ tự giảm dần**
  -   Neo4j:<br>
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool)<-[:Use_tool]-(pl) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework)<-[:Have_framework]-(pl) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform)-[Deploy_on_framework]->(fw) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return pl, kl, fw, tl, pf, sum(f.totalJobPost) as NumberOfJob order by NumberOfJob desc
  -   mssql:<br>
wait


In [1]:
import pandas as pd
from neo4j import GraphDatabase
import csv
import sparknlp as sp

uri = "bolt://localhost:7687"  # Adjust the URI based on your Neo4j server configuration
username = "neo4j"
password = "12345678"

# Create a Neo4j driver instance
driver = GraphDatabase.driver(uri, auth=(username, password))
driver

def queryToDataFrame(query):

    with driver.session() as session:
        result = session.run(query)
        columns = result.keys()
        data = [record for record in result]

    return pd.DataFrame(data, columns=columns)


In [3]:
query = f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)-[:Posted_on]->(w:Website) where w.name = "coursera" return c
'''
print("QUERY 1: ")
data = queryToDataFrame(query)
len(data)

303

In [17]:
query = f'''
match(f:FactJobPosting)-[:Belong_to_career]->(c:Career) where c.name = "Data Engineer" return sum(f.totalJobPost) as JobCount
'''
print("QUERY 2: ")

data = queryToDataFrame(query)
data.head(10)

Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))


QUERY 2: 


ServiceUnavailable: Failed to write data to connection IPv4Address(('localhost', 7687)) (ResolvedIPv4Address(('127.0.0.1', 7687)))

In [19]:
query = '''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
WHERE 
  EXISTS {(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) WHERE p.programmingLanguage IN ["JAVA", "PYTHON"]} 
  or EXISTS {(f)-[:Taught_framework]->(fw:Framework) WHERE fw.framework IN ["SPRING", "SPARK"]}
  or exists {(f)-[:Taught_tool]->(tl:Tool) where tl.tool = "GIT"}
  or exists {(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]}
  or exists {(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]}
RETURN count(DISTINCT c)
'''
print("QUERY 3: ")

data = queryToDataFrame(query)
len(data)
data.head(10)

QUERY 3: 


,count(DISTINCT c)
0,16


In [13]:
query = f'''
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool)<-[:Use_tool]-(pl) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework)<-[:Have_framework]-(pl) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform)-[Deploy_on_framework]->(fw) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name as CareerName,
    pl.programmingLanguage as ProgrammingLanguage,
tl.tool, kl.knowledge, fw.framework, pf.platform
'''

print("QUERY 4: ")

data = queryToDataFrame(query)
print(len(data))
data.head(10)

190


,CareerName,ProgrammingLanguage,tl.tool,kl.knowledge,fw.framework,pf.platform
0,Mobile Developer,JAVA,GIT,None,None,None
1,Devops Engineer,JAVA,GIT,None,None,None
2,Data Engineer,JAVA,GIT,None,SPARK,None
3,Frontend Developer,JAVA,GIT,None,SPRING,None
4,Mobile Developer,JAVA,GIT,None,None,None
5,Mobile Developer,JAVA,GIT,None,None,None
6,Data Analysts,PYTHON,None,DATA WAREHOUSE,None,None
7,Business Analyst,PYTHON,None,DATA ANALYSIS,None,None
8,Database Administrator,PYTHON,None,None,None,None
9,Data Engineer,PYTHON,None,None,SPARK,None


In [20]:
query = f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Taught_framework]->(fw:Framework)  where fw.framework in ["SPRING", "SPARK"]
optional match(f)-[:Taught_tool] ->(tl:Tool) where tl.tool = "GIT"
optional match(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return p.programmingLanguage, tl.tool, kl.knowledge, fw.framework, pf.platform, count(f) as NumberOfCourse order by NumberOfCourse desc 
'''

print("QUERY 5: ")
data = queryToDataFrame(query)
print(len(data))

data.head(10)

QUERY 5: 
2


,p.programmingLanguage,tl.tool,kl.knowledge,fw.framework,pf.platform,NumberOfCourse
0,JAVA,None,None,None,None,11
1,PYTHON,None,None,None,None,3


In [15]:
query = f'''
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return c.name, pl.programmingLanguage, kl.knowledge, fw.framework, tl.tool, pf.platform, sum(f.totalJobPost) as NumberOfJob order by NumberOfJob desc
'''
print("QUERY 6: ")

data = queryToDataFrame(query)
print(len(data))
data.head(10)

QUERY 6: 
127


,c.name,pl.programmingLanguage,kl.knowledge,fw.framework,tl.tool,pf.platform,NumberOfJob
0,Data Analysts,PYTHON,None,None,None,None,12
1,Devops Engineer,PYTHON,None,None,None,CLOUD,11
2,Business Analyst,PYTHON,None,None,None,None,10
3,Data Engineer,PYTHON,None,SPARK,None,CLOUD,10
4,Network Engineer,PYTHON,None,None,None,CLOUD,10
5,Software Engineer,PYTHON,None,None,None,None,7
6,Devops Engineer,PYTHON,None,None,None,AZURE,6
7,Network Engineer,PYTHON,None,None,None,AZURE,6
8,Database Administrator,PYTHON,None,None,None,AZURE,6
9,Software Engineer,JAVA,None,None,None,None,6


In [16]:
query = '''
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career) 
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge)<-[:Relate_to_knowledge]-(pl) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool)<-[:Use_tool]-(pl) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework)<-[:Have_framework]-(pl) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform)<-[:Deploy_to_platform]-(fw) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return c.name, pl.programmingLanguage, kl.knowledge, fw.framework, tl.tool, pf.platform, sum(f.totalJobPost) as NumberOfJob order by NumberOfJob desc
'''
print("QUERY 6.1: ")

data = queryToDataFrame(query)
print(len(data))
data.head(10)

QUERY 6.1: 
50


,c.name,pl.programmingLanguage,kl.knowledge,fw.framework,tl.tool,pf.platform,NumberOfJob
0,Data Analysts,PYTHON,None,None,None,None,19
1,Data Engineer,PYTHON,None,SPARK,None,None,16
2,Devops Engineer,PYTHON,None,None,None,None,15
3,Network Engineer,PYTHON,None,None,None,None,15
4,Business Analyst,PYTHON,None,None,None,None,10
5,Database Administrator,PYTHON,None,None,None,None,9
6,Software Engineer,JAVA,None,None,None,None,9
7,Data Engineer,PYTHON,None,None,None,None,9
8,Data Engineer,JAVA,None,SPARK,None,None,9
9,Software Engineer,PYTHON,None,None,None,None,8


## Code phân chia file:
1MB, 64MB, 256MB, 1024MB

In [2]:
import pandas as pd
import csv
DEFAULT_PATH = "/Users/nguyenvanviet/Work/Courses/DataGen/RecommandationCourseWeb-Neo4j/MyPaperDataSet/ETLCode/GDWData/"


In [10]:
def divideFile(data, fileName, fileSize, fixSize=1200):
    lenght = len(data)
    size = int(lenght/fixSize * fileSize) # divide 1124 because default file size is 1,2 GB onl disk
    fileName = fileName + str(fileSize) + "MB.csv"
    dataToWrite = data.iloc[0:size]
    dataToWrite.to_csv(fileName, index=False)
    print("Done")
    



In [5]:
fileName = DEFAULT_PATH + "jobDataForRDW.csv"
data = pd.read_csv(fileName)
fileName = "DataSet/JobDataRDW"
divideFile(data, fileName, 1)
divideFile(data, fileName, 64)
divideFile(data, fileName, 256)


In [6]:
len(data)

9526368

In [46]:
len(data)

9526368

In [10]:


newData = data.drop_duplicates(["ProgrammingLanguage", "ID", "Career", "Organization", "Location", "Date", "Month", "Year"])

In [23]:

newDataS = newData[(newData["ProgrammingLanguage"]) == "JAVA"]
newDataS.groupby(["ProgrammingLanguage", "Career"]).sum("JobCount")


jobCount  Date  Month   Year  \
ProgrammingLanguage Career                                                 
JAVA                Backend Developer             10   126      9  18216   
                    Business Analyst               1    28     12   2023   
                    Data Analysts                  1    14      1   2024   
                    Data Engineer                 13   152     32  20238   
                    Data Scientist                 2     2      1   2024   
                    Database Administrator         6    70      5  10120   
                    Devops Engineer                3    42      3   6072   
                    Frontend Developer             5    56      4   8096   
                    Game Development               1    14      1   2024   
                    Mobile Developer               5    42      3   6072   
                    Software Architect            10   126      9  18216   
                    Software Engineer             10   112      8  16192   
                    Tester                         2    28      2   4048   
                    Web Development                4    56      4   8096   

                                               ID  
ProgrammingLanguage Career                         
JAVA                Backend Developer       45738  
                    Business Analyst         6110  
                    Data Analysts            6392  
                    Data Engineer           52399  
                    Data Scientist           5631  
                    Database Administrator  26466  
                    Devops Engineer         15391  
                    Frontend Developer      21764  
                    Game Development         4767  
                    Mobile Developer        16166  
                    Software Architect      46226  
                    Software Engineer       46014  
                    Tester                  10268  
                    Web Development         19846

In [12]:
fileName = DEFAULT_PATH + "courseDataForRDW.csv"
data = pd.read_csv(fileName)
fileName = "DataSet/courseDataForRDW"
print(len(data))
divideFile(data, fileName, 1, 5)
divideFile(data, fileName, 2, 5)
divideFile(data, fileName, 3, 5)


24185
Done
Done
Done


In [34]:

fileName = DEFAULT_PATH + "courseDataForRDW.csv"
data = pd.read_csv(fileName)

print(len(data))

24185


In [35]:
data.groupby("ID").describe()

Duration                                          Price       ... Enroll  \
        count  mean  std   min   25%   50%   75%   max count mean  ...    75%   
ID                                                                 ...          
0         6.0   9.0  0.0   9.0   9.0   9.0   9.0   9.0   6.0  0.0  ...   13.0   
4         7.0  32.0  0.0  32.0  32.0  32.0  32.0  32.0   7.0  0.0  ...    8.0   
7         6.0  19.0  0.0  19.0  19.0  19.0  19.0  19.0   6.0  0.0  ...    1.0   
12       12.0  37.0  0.0  37.0  37.0  37.0  37.0  37.0  12.0  0.0  ...   29.0   
18        3.0  12.0  0.0  12.0  12.0  12.0  12.0  12.0   3.0  0.0  ...    0.0   
...       ...   ...  ...   ...   ...   ...   ...   ...   ...  ...  ...    ...   
4132      1.0  14.0  NaN  14.0  14.0  14.0  14.0  14.0   1.0  0.0  ...    9.0   
4136      2.0  11.0  0.0  11.0  11.0  11.0  11.0  11.0   2.0  0.0  ...    0.0   
4141      1.0  26.0  NaN  26.0  26.0  26.0  26.0  26.0   1.0  0.0  ...   78.0   
4143      1.0   8.0  NaN   8.0   8.0   8.0   8.0   8.0   1.0  0.0  ...    0.0   
4147     10.0  15.0  0.0  15.0  15.0  15.0  15.0  15.0  10.0  0.0  ...   66.0   

            Rate                                              
       max count mean           std  min  25%  50%  75%  max  
ID                                                            
0     13.0   6.0  4.6  9.729507e-16  4.6  4.6  4.6  4.6  4.6  
4      8.0   7.0  4.1  9.593423e-16  4.1  4.1  4.1  4.1  4.1  
7      1.0   6.0  0.0  0.000000e+00  0.0  0.0  0.0  0.0  0.0  
12    29.0  12.0  4.0  0.000000e+00  4.0  4.0  4.0  4.0  4.0  
18     0.0   3.0  0.0  0.000000e+00  0.0  0.0  0.0  0.0  0.0  
...    ...   ...  ...           ...  ...  ...  ...  ...  ...  
4132   9.0   1.0  4.6           NaN  4.6  4.6  4.6  4.6  4.6  
4136   0.0   2.0  4.5  0.000000e+00  4.5  4.5  4.5  4.5  4.5  
4141  78.0   1.0  4.6           NaN  4.6  4.6  4.6  4.6  4.6  
4143   0.0   1.0  4.4           NaN  4.4  4.4  4.4  4.4  4.4  
4147  66.0  10.0  4.8  9.362223e-16  4.8  4.8  4.8  4.8  4.8  

[652 rows x 32 columns]

In [36]:
data.groupby("ProgrammingLanguage").describe()

Duration                                                \
                       count       mean       std    min   25%   50%   75%   
ProgrammingLanguage                                                          
AMAZON WEB SERVICES      4.0  12.500000  9.000000  -1.00  12.5  17.0  17.0   
AMBARI                  18.0  14.500000  0.000000  14.50  14.5  14.5  14.5   
ANDROID SDK             10.0  -1.000000  0.000000  -1.00  -1.0  -1.0  -1.0   
ANGULARJS               56.0   7.000000  0.000000   7.00   7.0   7.0   7.0   
ANSIBLE                  4.0  -1.000000  0.000000  -1.00  -1.0  -1.0  -1.0   
...                      ...        ...       ...    ...   ...   ...   ...   
WEBASSEMBLY              1.0  -1.000000       NaN  -1.00  -1.0  -1.0  -1.0   
WORDPRESS                2.0  -1.000000  0.000000  -1.00  -1.0  -1.0  -1.0   
WPF                      2.0  -1.000000  0.000000  -1.00  -1.0  -1.0  -1.0   
XAMARIN                  2.0  -1.000000  0.000000  -1.00  -1.0  -1.0  -1.0   
XML                    171.0   5.355088  2.761318   0.18   6.0   6.0   6.0   

                           Price             ...   Rate          ID  \
                      max  count       mean  ...    75%  max  count   
ProgrammingLanguage                          ...                      
AMAZON WEB SERVICES  17.0    4.0  -0.250000  ...  4.500  4.5    4.0   
AMBARI               14.5   18.0   9.990000  ...  4.600  4.6   18.0   
ANDROID SDK          -1.0   10.0  -1.000000  ...  4.325  4.7   10.0   
ANGULARJS             7.0   56.0  13.990000  ...  3.900  3.9   56.0   
ANSIBLE              -1.0    4.0  -1.000000  ...  3.800  3.8    4.0   
...                   ...    ...        ...  ...    ...  ...    ...   
WEBASSEMBLY          -1.0    1.0  -1.000000  ...  3.700  3.7    1.0   
WORDPRESS            -1.0    2.0  -1.000000  ...  4.500  4.5    2.0   
WPF                  -1.0    2.0  -1.000000  ...  4.000  4.0    2.0   
XAMARIN              -1.0    2.0  -1.000000  ...  4.400  4.4    2.0   
XML                  19.0  171.0  19.522573  ...  4.500  4.8  171.0   

                                                                              \
                            mean         std     min     25%     50%     75%   
ProgrammingLanguage                                                            
AMAZON WEB SERVICES  2204.500000  257.000000  1819.0  2204.5  2333.0  2333.0   
AMBARI               2308.000000    0.000000  2308.0  2308.0  2308.0  2308.0   
ANDROID SDK          1681.400000   50.328256  1588.0  1691.0  1698.0  1715.0   
ANGULARJS             552.000000    0.000000   552.0   552.0   552.0   552.0   
ANSIBLE              1511.000000    0.000000  1511.0  1511.0  1511.0  1511.0   
...                          ...         ...     ...     ...     ...     ...   
WEBASSEMBLY          1616.000000         NaN  1616.0  1616.0  1616.0  1616.0   
WORDPRESS            1678.000000    0.000000  1678.0  1678.0  1678.0  1678.0   
WPF                  1247.000000    0.000000  1247.0  1247.0  1247.0  1247.0   
XAMARIN              1241.000000    0.000000  1241.0  1241.0  1241.0  1241.0   
XML                   965.309942  305.798517   199.0   978.0   978.0   978.0   

                             
                        max  
ProgrammingLanguage          
AMAZON WEB SERVICES  2333.0  
AMBARI               2308.0  
ANDROID SDK          1715.0  
ANGULARJS             552.0  
ANSIBLE              1511.0  
...                     ...  
WEBASSEMBLY          1616.0  
WORDPRESS            1678.0  
WPF                  1247.0  
XAMARIN              1241.0  
XML                  2952.0  

[122 rows x 40 columns]

In [ ]:
q1 =f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)-[:Posted_on]->(w:Website) where w.name = "coursera" 
return c 

'''

q2 =f'''
match(f:FactJobPosting)-[:Belong_to_career]->(c:Career) where c.name = "Data Engineer" return sum(f.totalJobPost) as JobCount



'''

q3 =f'''

match(c:Course)<-[:Belong_to_course]-(f:FactCourse) where exists {(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]} or exists {(f)-[:Taught_framework]->(fw:Framework) where fw.framework in ["SPRING", "SPARK"]} or exists {(f)-[:Taught_tool]->(tl:Tool) where tl.tool = "GIT"} or exists {(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]} or exists {(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]}
return distinct c
'''

q4 =f'''

match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name as CareerName,
 pl.programmingLanguage as ProgrammingLanguage,
  kl.knowledge as Knowledge,   fw.framework as Framework,      tl.tool as Tool, 
pf.platform as Platform
'''

q5 =f'''
match(c:Course)<-[:Belong_to_course]-(f:FactCourse)
 match(f)-[:Taught_programmingLanguage]->(p:ProgrammingLanguage) where p.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Taught_framework]->(fw:Framework) where fw.framework in ["SPRING", "SPARK"]
optional match(f)-[:Taught_tool]->(tl:Tool) where tl.tool = "GIT"
optional match(f)-[:Taught_knowledge]->(kl:Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Taught_platform]->(pf:Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]
return p, tl, kl, fw, pf, count(f) as NumberOfCourse order by NumberOfCourse desc
'''

q6 =f'''
match(f: FactJobPosting)-[:Belong_to_career]->(c: Career)
match(f)-[:Required_programmingLanguage]->(pl : ProgrammingLanguage) where pl.programmingLanguage in ["JAVA", "PYTHON"]
optional match(f)-[:Required_knowledge]->(kl: Knowledge) where kl.knowledge in ["DATA ANALYSIS", "DATA WAREHOUSE"]
optional match(f)-[:Required_tool]->(tl: Tool) where tl.tool = "GIT"
optional match(f)-[:Required_framework]->(fw: Framework) where fw.framework in [ "SPARK", "SPRING"]
optional match(f)-[:Required_platform]->(pf: Platform) where pf.platform in ["AZURE", "CLOUD", "MYSQL"]

return c.name, pl.programmingLanguage, kl, fw.framework, tl, pf, sum(f.totalJobPost) as NumberOfJob order by NumberOfJob desc
'''